In [4]:
import os
import numpy as np
import pandas as pd
from skimage import io, measure, transform
from cellpose import models
import matplotlib.pyplot as plt
from ipywidgets import Button, IntText, FloatProgress, Output, VBox
from ipyfilechooser import FileChooser
from IPython.display import clear_output

# Set up Cellpose model
model = models.Cellpose(model_type="cyto3")

# Function to create mask overlay
def create_mask_overlay(image, mask):
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.imshow(image, cmap="gray")
    ax.imshow(mask, alpha=0.5, cmap="hsv")
    ax.axis("off")
    return fig


# Function to process images
def process_images(folder_path, progress_bar, output_area):
    results = []
    overlay_dir = os.path.join(folder_path, "mask_overlays")
    os.makedirs(overlay_dir, exist_ok=True)

    image_files = [
        f
        for f in os.listdir(folder_path)
        if f.lower().endswith((".tif", ".tiff", ".png"))
    ]
    total_images = len(image_files)

    for i, filename in enumerate(image_files):
        img_path = os.path.join(folder_path, filename)
        img = io.imread(img_path)
        img = transform.resize(img, (int(img.shape[0] * 0.5), int(img.shape[1] * 0.5)))

        try:
            masks, flow, styles = models.CellposeModel(model_type="cyto3").eval(
                img, diameter=(diameter.value / 2), channels=[0, 0], flow_threshold=0.4
            )
        except ValueError as e:
            with output_area:
                print(f"Error processing {filename}: {str(e)}")
            continue

        progress_bar.value = (i + 1) / total_images
        with output_area:
            output_area.clear_output(wait=True)
            print(f"Processed image {i+1} of {total_images}: {filename}")

        fig = create_mask_overlay(img, masks)
        overlay_path = os.path.join(
            overlay_dir, f"{os.path.splitext(filename)[0]}_overlay.png"
        )
        fig.savefig(overlay_path, dpi=300, bbox_inches="tight")
        with output_area:
            display(fig)
        plt.close(fig)

        props = measure.regionprops(masks)
        for j, prop in enumerate(props):
            results.append(
                {
                    "filename": filename,
                    "roi_index": j,
                    "diameter": prop.equivalent_diameter,
                    "area": prop.area,
                    "major_axis_length": prop.major_axis_length,
                    "minor_axis_length": prop.minor_axis_length,
                    "aspect_ratio": (
                        prop.major_axis_length / prop.minor_axis_length
                        if prop.minor_axis_length != 0
                        else np.nan
                    ),
                }
            )

    df = pd.DataFrame(results)
    filtered_df = df[(df["aspect_ratio"] >= 0.8) & (df["aspect_ratio"] <= 1.2)]
    return filtered_df


# Set up GUI components
folder_chooser = FileChooser(
    title="Select the folder containing your images", show_only_dirs=True
)
diameter = IntText(value=100, description="Est. diam.")
process_button = Button(description="Process Images")
progress_bar = FloatProgress(min=0, max=1, description="Progress:", bar_style="info")
output_area = Output()


# Define button click event


def on_button_clicked(b):
    folder_path = folder_chooser.selected_path
    if not folder_path or not os.path.isdir(folder_path):
        with output_area:
            print("Please select a valid folder.")
        return

    with output_area:
        print(f"Processing images in {folder_path}...")

    results_df = process_images(folder_path, progress_bar, output_area)

    if results_df.empty:
        print(
            "No results to export. Please check if there were any errors during processing and if you have images on the selected folder."
        )
        return

    output_path = os.path.join(folder_path, "segmentation_results.xlsx")
    results_df.to_excel(output_path, index=False)
    print(f"Results exported to {output_path}")
    print(f"Mask overlays saved in {os.path.join(folder_path, 'mask_overlays')}")


process_button.on_click(on_button_clicked)

# Display GUI components
display(VBox([folder_chooser, diameter, process_button, progress_bar, output_area]))